# Denoising experiments

## Import libraries

In [6]:
# Import necessary libraries
import mne
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Load the data
file_path = r"Raw Data - 02052024\OpenBCISession_2024-02-09_11-10-01\OpenBCI-RAW-2024-02-09_11-31-53.txt"
df = pd.read_csv(file_path, comment='%')
df = df.rename(columns=lambda x: x.strip())
df = df[5:]  # Remove unnecessary header rows
df = df.reset_index(drop=True)  # Reset indices

# Display the first few rows to inspect
print(df.head())

In [1]:


# Step 2: Create RawArray for signal processing
# Selecting EEG data columns and setting sampling frequency
data_ica_df = df.iloc[:, 1:9]  # Assuming EEG data is in the first 8 columns after the index
sfreq = 125  # Sampling frequency in Hz
info = mne.create_info(ch_names=data_ica_df.columns.tolist(), sfreq=sfreq, ch_types='eeg')

# Create RawArray for EEG data
raw_eeg = mne.io.RawArray(data_ica_df.T.values * 1e-6, info)  # Convert µV to V

# Create EOG channel
eog_channel = data_ica_df.iloc[:, 0].values * 1e-6  # Convert µV to V
eog_info = mne.create_info(['EOG'], sfreq=sfreq, ch_types=['eog'])
eog_raw = mne.io.RawArray(eog_channel[None, :], eog_info)

# Merge EEG and EOG data before filtering
raw_combined = raw_eeg.add_channels([eog_raw])

# Apply bandpass filter to the combined data
raw_combined.filter(l_freq=1.0, h_freq=40.0, fir_design='firwin')

# Step 3: Create EOG epochs to detect blinks
eog_epochs = mne.preprocessing.create_eog_epochs(raw_combined, ch_name='EOG')  # Specify the new EOG channel
eog_events = eog_epochs.events

# Print and create DataFrame for events
print(eog_events)
eog_events_df = pd.DataFrame(eog_events, columns=['sample', 'prev_event_id', 'event_id'])

# Step 4: Visualize raw and filtered data with detected blinks
fig, axs = plt.subplots(len(data_ica_df.columns), 2, figsize=(15, 3 * len(data_ica_df.columns)))

for i, channel in enumerate(data_ica_df.columns):
    # Raw signal display
    axs[i, 0].plot(df.index / sfreq, df[channel], label='Raw Data')
    axs[i, 0].set_title(f'Raw Data (Channel {i})')
    axs[i, 0].set_ylabel('Amplitude (µV)')
    if i == len(data_ica_df.columns) - 1:
        axs[i, 0].set_xlabel('Time (s)')

    # Filtered signal display
    axs[i, 1].plot(raw_combined.times, raw_combined.get_data(picks=[i])[0] * 1e6, label='Filtered Data')  # Convert back to µV
    
    # Mark detected blinks on the filtered data
    blink_samples = eog_events_df['sample'].values
    blink_times = blink_samples / sfreq
    for blink_time in blink_times:
        axs[i, 1].axvline(x=blink_time, color='r', linestyle='--', label='Detected Blink' if blink_time == blink_times[0] else "")
    
    axs[i, 1].set_title(f'Filtered Data (Channel {i})')
    axs[i, 1].set_ylabel('Amplitude (µV)')
    if i == len(data_ica_df.columns) - 1:
        axs[i, 1].set_xlabel('Time (s)')
    axs[i, 1].legend()

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Display the plot with detected events
plt.show()

print(eog_events_df)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn

## Step 1: Data Analysis

In [2]:
df = pd.read_csv(r"D:\Code\NEvol_git\Sandbox\Raw Data - 02052024\OpenBCISession_2024-02-09_11-10-01\OpenBCI-RAW-2024-02-09_11-31-53.txt", comment='%')
df = df.rename(columns=lambda x: x.strip())
df = df[5:]
df.head()

In [3]:
df.columns

In [4]:
def plot_eeg(data, figsize=(20, 15), title=None):
    if isinstance(data, torch.Tensor):
        data = data.detach().numpy()

    num_plots = data.shape[1]

    fig, ax = plt.subplots(num_plots, 1, figsize=figsize)
    x = range(len(data))

    if data.ndim == 1:
        data = data[:, None]

    if num_plots == 1:
        ax = [ax]
        # data = data.reshape(1, -1)

    for i in range(num_plots):
        ax[i].plot(x, data[:, i], linewidth=0.5)
        ax[i].set_xlabel("Iter")
        ax[i].set_ylabel("Voltage ($\mu V$)")
        if title:
            ax[i].set_title(title)
        else:
            ax[i].set_title(df.columns[start_idx+i])

    plt.tight_layout()
    plt.show()

In [5]:
start_idx = 1
end_idx = start_idx + 8
raw_data = df.iloc[:, start_idx:end_idx].to_numpy()
plot_eeg(raw_data)

## 1. 1D Convolutions

In [6]:
data_conv_df = df.iloc[:, 1]
data_conv_df.head()

In [7]:
data_conv = torch.tensor(data_conv_df.to_numpy(), dtype=torch.float32).view(1, 1, -1)

In [8]:
class ConvolutionNoiseReductor(nn.Module):
    def __init__(self, kernel_size, stride=None, padding=0):
        super(ConvolutionNoiseReductor, self).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, stride=stride, padding=padding)

    def forward(self, x):
        x = self.conv(x)
        return x

In [9]:
model = ConvolutionNoiseReductor(kernel_size=256, stride=16)

result = model(data_conv)

In [10]:
plot_eeg(df["EXG Channel 0"].to_numpy()[..., np.newaxis], figsize=(15, 3), title="Original")
plot_eeg(result[0, :, :].T, figsize=(15, 3), title="Denoinsed")

**Convolutions ❌** \\
One major flaw of using convolutions for EEG signal denoising is that the values have a very large shift after applying the convolution.

## 2. ICA

In [11]:
!pip install -q mne

In [12]:
import mne
from mne.preprocessing import ICA

In [13]:
data_ica_df = df.iloc[:, 1:9]
data_ica_df.head()

In [14]:
sfreq = 125
info = mne.create_info(ch_names=data_ica_df.columns.tolist(), sfreq=sfreq, ch_types='eeg')

raw = mne.io.RawArray(data_ica_df.T.values, info)

#ica = ICA(n_components=None, random_state=42, max_iter=800)
ica = ICA(n_components=0.95, random_state=42, max_iter=800)

#raw.filter(l_freq=1.0, h_freq=None)
raw.filter(l_freq=1.0, h_freq=40)
ica.fit(raw)

In [15]:
ica.apply(raw)
denoised_df = pd.DataFrame(raw.get_data().T, columns=data_ica_df.columns.tolist())
denoised_df.head()

In [16]:
plot_eeg(df["EXG Channel 0"].to_numpy()[..., np.newaxis], figsize=(15, 3), title="Original")
plot_eeg(denoised_df.to_numpy(), title="Denoinsed")

In [17]:
#filtering experiment from below

In [18]:
denoised_df

In [25]:
import numpy as np
import pandas as pd
from scipy import signal
from mne import create_info, EpochsArray
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Sampling rate
sampling_rate = 125

# Bandpass filter
low_cut = 0.5  # Low cutoff frequency (Hz)
high_cut = 30  # High cutoff frequency (Hz)
nyquist = sampling_rate / 2
low = low_cut / nyquist
high = high_cut / nyquist
order = 4  # Filter order
normalized_cutoff = [low, high]
filter_coefficients = signal.butter(order, normalized_cutoff, btype='band', output='ba')

# Notch filter
notch_freq = 50  # Notch frequency (Hz) to remove power line noise
notch_quality_factor = 30  # Quality factor for the notch filter
notch_filter = signal.iirnotch(notch_freq, notch_quality_factor, sampling_rate)

# Apply filters and save filtered data to a new DataFrame
filtered_df = denoised_df.copy()
for col in filtered_df.columns:
    filtered_df[col] = signal.filtfilt(*filter_coefficients, filtered_df[col])
    filtered_df[col] = signal.filtfilt(*notch_filter, filtered_df[col])

# Re-reference to average
filtered_df = filtered_df.sub(filtered_df.mean(axis=1), axis=0)

# Baseline correction
baseline_start = 0  # Start time (in seconds) for baseline
baseline_end = 1  # End time (in seconds) for baseline
baseline_samples = slice(int(baseline_start * sampling_rate), int(baseline_end * sampling_rate))
baseline_mean = filtered_df.iloc[baseline_samples].mean()
filtered_df = filtered_df.sub(baseline_mean, axis=1)

# Imagined speech epoch extraction
epoch_duration = 1  # Duration of each epoch in seconds
epoch_start_time = 3  # Start time of epoch extraction in seconds
epoch_end_time = 10  # End time of epoch extraction in seconds
epoch_samples_per_epoch = int(epoch_duration * sampling_rate)

epochs_data_filtered = []
epochs_data_denoised = []

for i in range(len(filtered_df) // epoch_samples_per_epoch):
    epoch_data_filtered = filtered_df.iloc[i * epoch_samples_per_epoch : (i + 1) * epoch_samples_per_epoch].values.T
    epochs_data_filtered.append(epoch_data_filtered)

    # Assuming denoised_data is available with the same shape as filtered_data
    epoch_data_denoised = denoised_df.iloc[i * epoch_samples_per_epoch : (i + 1) * epoch_samples_per_epoch].values.T
    epochs_data_denoised.append(epoch_data_denoised)

# Create MNE info object
n_channels = len(filtered_df.columns)
ch_names = filtered_df.columns.tolist()
ch_types = ['eeg'] * n_channels
info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sampling_rate)

# Create MNE EpochsArray for filtered data
epochs_filtered = EpochsArray(epochs_data_filtered, info)

# Create MNE EpochsArray for denoised data
epochs_denoised = EpochsArray(epochs_data_denoised, info)

# Plotting all channels
num_channels = len(epochs_filtered.ch_names)
fig, axs = plt.subplots(num_channels, 2, figsize=(10, 6*num_channels), sharex=True)

for i in range(num_channels):
    axs[i, 0].plot(epochs_filtered.times, epochs_filtered.get_data()[:, i, :].mean(axis=0))
    axs[i, 0].set_title(f'Filtered Data (Channel {i})')
    axs[i, 0].set_ylabel('Amplitude (μV)')

    axs[i, 1].plot(epochs_denoised.times, epochs_denoised.get_data()[:, i, :].mean(axis=0))
    axs[i, 1].set_title(f'Denoised Data (Channel {i})')
    axs[i, 1].set_ylabel('Amplitude (μV)')

# Set x-axis label only for the last subplot
axs[-1, 0].set_xlabel('Time (s)')
axs[-1, 1].set_xlabel('Time (s)')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Display the plot
plt.show()


In [26]:
# Calculate mean of filtered EEG data for each channel
mean_data = filtered_df.mean()

# Create a new DataFrame to store the mean data
mean_df = pd.DataFrame(mean_data, columns=['Mean Amplitude (μV)'])

# Display the new DataFrame
print(mean_df)


In [27]:
filtered_df.to_csv("filtered_data.csv", index=False)


In [28]:
denoised_df

In [29]:
filtered_df

In [37]:
# Загрузка и создание объекта `raw`
data_ica_df = df.iloc[:, 1:9]
sfreq = 125
info = mne.create_info(ch_names=data_ica_df.columns.tolist(), sfreq=sfreq, ch_types='eeg')
raw = mne.io.RawArray(data_ica_df.T.values, info)

# Теперь raw_df содержит исходные данные
raw_df = pd.DataFrame(data=raw.get_data().T, columns=data_ica_df.columns.tolist())

# Количество каналов
n_channels = len(raw_df.columns)
fig, axs = plt.subplots(n_channels, 2, figsize=(15, 3 * n_channels))

for i, channel in enumerate(raw_df.columns):
    # Исходный сигнал (raw)
    axs[i, 0].plot(raw_df.index / sampling_rate, raw_df[channel])
    axs[i, 0].set_title(f'Raw Data (Channel {i})')
    axs[i, 0].set_ylabel('Amplitude (µV)')
    if i == n_channels - 1:
        axs[i, 0].set_xlabel('Time (s)')

    # Фильтрованный сигнал (filtered)
    axs[i, 1].plot(filtered_df.index / sampling_rate, filtered_df[channel])
    axs[i, 1].set_title(f'Filtered Data (Channel {i})')
    axs[i, 1].set_ylabel('Amplitude (µV)')
    if i == n_channels - 1:
        axs[i, 1].set_xlabel('Time (s)')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Отображаем график
plt.show()

In [39]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Предполагаем, что отфильтрованные данные загружены в объект RawArray
filtered_data = filtered_df.T.values * 1e-6  # конвертация в вольты, если данные в микровольтах
info = mne.create_info(ch_names=filtered_df.columns.tolist(), sfreq=sampling_rate, ch_types=['eeg']*n_channels)
raw_filtered = mne.io.RawArray(filtered_data, info)

# Создание EOG каналов из одного из уже существующих каналов (например, первый канал)
raw_filtered.set_channel_types({channel: 'eog' for channel in filtered_df.columns[0:1]})

# Создание EOG epochs для обнаружения морганий
eog_epochs = mne.preprocessing.create_eog_epochs(raw_filtered, ch_name=filtered_df.columns[0])
eog_events = eog_epochs.events

# Печать и создание DataFrame для событий
print(eog_events)
eog_events_df = pd.DataFrame(eog_events, columns=['sample', 'prev_event_id', 'event_id'])

# Графики сигналов и отмеченных EOG событий
fig, axs = plt.subplots(n_channels, 2, figsize=(15, 3 * n_channels))

for i, channel in enumerate(filtered_df.columns):
    # Исходный сигнал (raw)
    axs[i, 0].plot(raw_df.index / sampling_rate, raw_df[channel], label='Raw Data')
    axs[i, 0].set_title(f'Raw Data (Channel {i})')
    axs[i, 0].set_ylabel('Amplitude (µV)')
    if i == n_channels - 1:
        axs[i, 0].set_xlabel('Time (s)')

    # Фильтрованный сигнал (filtered)
    axs[i, 1].plot(filtered_df.index / sampling_rate, filtered_df[channel], label='Filtered Data')
    
    # Отметим события на графиках
    blink_samples = eog_events_df['sample'].values
    blink_times = blink_samples / sampling_rate
    for blink_time in blink_times:
        axs[i, 1].axvline(x=blink_time, color='r', linestyle='--', label='Detected Blink' if blink_time == blink_times[0] else "")
    
    axs[i, 1].set_title(f'Filtered Data (Channel {i})')
    axs[i, 1].set_ylabel('Amplitude (µV)')
    if i == n_channels - 1:
        axs[i, 1].set_xlabel('Time (s)')
    axs[i, 1].legend()

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Отображаем график с отмеченными событиями
plt.show()

print(eog_events_df)